In [1]:
import json


In [2]:
with open ('../data/finetune_data.json') as f:
    data = json.load(f)

In [3]:
mat = data['mp-644256']
mat.keys()

dict_keys(['incar', 'kpoints', 'poscar', 'robocrys', 'structure'])

In [4]:
mat['poscar']

'Li4 H8 C4 O12\n1.0\n   4.9024999999999999    0.0000000000000000    0.0000000000000000\n   0.0000000000000000    6.5645600000000002    0.0000000000000000\n   0.0000000000000000    0.0000000000000000    9.9938570000000002\nLi H C O\n4 8 4 12\ndirect\n   0.2413980000000000    0.3816140000000000    0.4506720000000000 Li\n   0.7413980000000000    0.6183860000000000    0.5493280000000000 Li\n   0.7413980000000000    0.8816140000000000    0.0493280000000000 Li\n   0.2413980000000000    0.1183860000000000    0.9506720000000000 Li\n   0.6417090000000000    0.3318130000000000    0.2149830000000000 H\n   0.1417090000000000    0.6681870000000000    0.7850170000000000 H\n   0.1417090000000000    0.8318130000000000    0.2850170000000000 H\n   0.6417090000000000    0.1681870000000000    0.7149830000000000 H\n   0.1258510000000000    0.4014190000000000    0.1291760000000000 H\n   0.6258510000000000    0.5985810000000000    0.8708240000000000 H\n   0.6258510000000000    0.9014190000000000    0.3708240

In [5]:
from tqdm import tqdm

In [6]:
res = []

for k, v in tqdm(data.items()):
    temp = {}
    temp['instruction'] = 'Generate the POSCAR file for the given crystal structure.'
    temp['input'] = v['robocrys']
    temp['output'] = v['poscar']
    res.append(temp)


100%|██████████| 1523/1523 [00:00<00:00, 206702.21it/s]


In [10]:
with open('test_POSCAR', 'w') as f:
    f.write(res[0]['output'])


In [ ]:
# train test split
from sklearn.model_selection import train_test_split

In [37]:
# train-val-test split approx 8-1-1
train, test = train_test_split(res, test_size=0.1, random_state=42)
train, val = train_test_split(train, test_size=0.1, random_state=42)

len(train), len(val), len(test)

NameError: name 'train_test_split' is not defined

In [ ]:
annotation = {
    'train': train,
    'validation': val,
    'test': test
}

In [ ]:
with open('../data/finetune_annotation.json', 'w') as f:
    json.dump(annotation, f)

# prepare few-shot prompts

In [38]:
with open ('../data/finetune_annotation.json') as f:
    annotations_all = json.load(f)

train_set = annotations_all['train']
val_set = annotations_all['validation']
test_set = annotations_all['test']

len(train_set), len(val_set), len(test_set)

(1233, 137, 153)

In [ ]:
train_set

In [42]:
train_set[0]['output']

'Li4 Ga4 Se8 O24\n1.0\n  -5.3669739999999999    5.3669739999999999    5.0097290000000001\n   5.3669739999999999   -5.3669739999999999    5.0097290000000001\n   5.3669739999999999    5.3669739999999999   -5.0097290000000001\nLi Ga Se O\n4 4 8 24\ndirect\n   0.6892280000000000    0.8750000000000000    0.3142280000000000 Li\n   0.5607720000000000    0.3750000000000000    0.6857720000000000 Li\n   0.6250000000000000    0.3107720000000000    0.1857720000000000 Li\n   0.1250000000000000    0.4392280000000000    0.8142280000000000 Li\n   0.4126740000000000    0.8750000000000000    0.0376740000000000 Ga\n   0.8373260000000000    0.3750000000000000    0.9623260000000000 Ga\n   0.6250000000000000    0.5873260000000000    0.4623260000000000 Ga\n   0.1250000000000000    0.1626740000000000    0.5376740000000000 Ga\n   0.8557700000000000    0.1345510000000000    0.1717520000000000 Se\n   0.2872010000000000    0.6154490000000000    0.2212190000000000 Se\n   0.9627990000000000    0.6840180000000000   

In [26]:
import random

In [41]:
! open .

In [28]:
instruction = "Generate the poscar file based on the description of a material. Here are some examples you can learn. In each example, <Example> denote this is an example, [Description] denotes the following texts are the description of a material, [POSCAR] denotes the following texts are the corresponding poscar file. <Question> denotes this is a question, and the subsequent texts are the description of a material. You need to generate the corresponding poscar file."

# each example should be string like this
# <Example>
# [Description]
# the description from res[i]['input']
# [POSCAR]
# the poscar from res[i]['output']
examples = []

# randomly select 10 examples
index = random.sample(range(len(res)), 15)

for i in index[:-2]:
    example = "<Example>\n"
    example += "[Description]\n"
    example += res[i]['input']
    example += "\n[POSCAR]\n"
    example += res[i]['output']
    examples.append(example)

question = "<Question>\n[Description]\n"
question += res[index[-1]]['input']

prompt = instruction + '\n' + '\n'.join(examples) + '\n' + question

prompt

'Generate the poscar file based on the description of a material. Here are some examples you can learn. In each example, <Example> denote this is an example, [Description] denotes the following texts are the description of a material, [POSCAR] denotes the following texts are the corresponding poscar file. <Question> denotes this is a question, and the subsequent texts are the description of a material. You need to generate the corresponding poscar file.\n<Example>\n[Description]\nLi₂Al₂Si₄HO₁₄ crystallizes in the triclinic P1 space group. There are two inequivalent Li sites. In the first Li site, Li is bonded to four O atoms to form LiO₄ tetrahedra that share corners with three AlO₄ tetrahedra and corners with three SiO₄ tetrahedra. There are a spread of Li-O bond distances ranging from 1.94-2.08 Å. In the second Li site, Li is bonded to four O atoms to form LiO₄ tetrahedra that share corners with three AlO₄ tetrahedra and corners with three SiO₄ tetrahedra. There are a spread of Li-O 

In [29]:
with open('prompt', 'w') as f:
    f.write(prompt)


In [30]:
with open('test_POSCAR', 'w') as f:
    f.write(res[index[-1]]['output'])

In [31]:
res[index[-1]]['input']

'LiHSeO₃ crystallizes in the orthorhombic P2_12_12₁ space group. Li¹⁺ is bonded to four O²⁻ atoms to form corner-sharing LiO₄ tetrahedra. There are a spread of Li-O bond distances ranging from 1.96-2.05 Å. H¹⁺ is bonded in a distorted linear geometry to two O²⁻ atoms. There is one shorter (1.02 Å) and one longer (1.60 Å) H-O bond length. Se⁴⁺ is bonded in a trigonal non-coplanar geometry to three O²⁻ atoms. There are a spread of Se-O bond distances ranging from 1.69-1.84 Å. There are three inequivalent O²⁻ sites. In the first O²⁻ site, O²⁻ is bonded in a trigonal planar geometry to one Li¹⁺, one H¹⁺, and one Se⁴⁺ atom. In the second O²⁻ site, O²⁻ is bonded in a trigonal planar geometry to two equivalent Li¹⁺ and one Se⁴⁺ atom. In the third O²⁻ site, O²⁻ is bonded in a 3-coordinate geometry to one Li¹⁺, one H¹⁺, and one Se⁴⁺ atom.'

In [34]:
structure = "Li4 Y2 P2 C2 O14\n1.0\n   6.6273569999999998    0.0000000000000000    0.0000000000000000\n   0.0000000000000000    4.9545070000000000    0.0000000000000000\n   0.0000000000000000    0.0000000000000000    9.6711609999999996\nLi Y P C O\n4 2 2 2 14\ndirect\n   0.7500000000000000    0.0000000000000000    0.0000000000000000 Li\n   0.2500000000000000    0.5000000000000000    0.5000000000000000 Li\n   0.2500000000000000    0.5000000000000000    0.0000000000000000 Li\n   0.7500000000000000    0.0000000000000000    0.5000000000000000 Li\n   0.2500000000000000    0.0000000000000000    0.0000000000000000 Y\n   0.7500000000000000    0.0000000000000000    0.0000000000000000 Y\n   0.2500000000000000    0.5000000000000000    0.5000000000000000 P\n   0.7500000000000000    0.0000000000000000    0.5000000000000000 P\n   0.2500000000000000    0.5000000000000000    0.0000000000000000 C\n   0.7500000000000000    0.0000000000000000    0.5000000000000000 C\n   0.2500000000000000    0.0000000000000000    0.0000000000000000 O\n   0.7500000000000000    0.7500000000000000    0.0000000000000000 O\n   0.2500000000000000    0.2500000000000000    0.0000000000000000 O\n   0.7500000000000000    0.2500000000000000    0.5000000000000000 O\n   0.2500000000000000    0.7500000000000000    0.5000000000000000 O\n   0.7500000000000000    0.2500000000000000    0.2500000000000000 O\n   0.2500000000000000    0.7500000000000000    0.7500000000000000 O\n   0.2500000000000000    0.0000000000000000    0.7500000000000000 O\n   0.7500000000000000    0.5000000000000000    0.2500000000000000 O\n   0.2500000000000000    0.5000000000000000    0.7500000000000000 O\n   0.7500000000000000    0.7500000000000000    0.5000000000000000 O\n   0.2500000000000000    0.2500000000000000    0.0000000000000000 O"

with open('llm_test_POSCAR', 'w') as f:
    f.write(structure)

In [35]:
structure = "Li4 Y2 P2 C2 O14 1.0 6.8062540000000000 0.0000000000000000 0.0000000000000000 0.0000000000000000 4.9153140000000004 0.0000000000000000 0.0000000000000000 0.4901310000000000 9.0731909999999996 Li Y P C O 4 2 2 2 14 direct 0.5208220000000000 0.7988310000000000 0.7975620000000000 Li 0.9791780000000000 0.7988310000000000 0.7975620000000000 Li 0.4791780000000000 0.2011690000000000 0.2024380000000000 Li 0.0208220000000000 0.2011690000000000 0.2024380000000000 Li 0.2500000000000000 0.2547240000000000 0.6373180000000001 Y 0.7500000000000000 0.7452760000000000 0.3626820000000000 Y 0.7500000000000000 0.2763260000000000 0.5925070000000000 P 0.2500000000000000 0.7236740000000000 0.4074930000000000 P 0.2500000000000000 0.2600530000000000 0.9401460000000000 C 0.7500000000000000 0.7399470000000000 0.0598540000000000 C 0.7500000000000000 0.7238420000000000 0.9201600000000000 O 0.2500000000000000 0.0260600000000000 0.8768190000000000 O 0.2500000000000000 0.4789140000000000 0.8499949999999999 O 0.9342190000000000 0.1388130000000000 0.6621280000000000 O 0.5657810000000000 0.1388130000000000 0.6621280000000000 O 0.2500000000000000 0.7549480000000000 0.5796210000000001 O 0.7500000000000000 0.5888830000000000 0.6091280000000000 O 0.2500000000000000 0.4111170000000000 0.3908720000000000 O 0.7500000000000000 0.2450520000000000 0.4203790000000000 O 0.4342190000000000 0.8611870000000000 0.3378720000000000 O 0.0657810000000000 0.8611870000000000 0.3378720000000000 O 0.7500000000000000 0.5210860000000000 0.1500050000000000 O 0.7500000000000000 0.9739400000000000 0.1231810000000000 O 0.2500000000000000 0.2761580000000000 0.0798400000000000 O"

with open('test_POSCAR', 'w') as f:
    f.write(structure)

In [26]:
# with open('../data/poscar_train.json', 'w') as f:
#     json.dump(train, f)

# with open('../data/poscar_val.json', 'w') as f:
#     json.dump(val, f)